<a href="https://colab.research.google.com/github/GiX007/agent-labs/blob/main/04_langGraph/00_simple_agent_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple ReAct Agent from Scratch

In [ ]:
# based on https://til.simonwillison.net/llms/python-react-pattern

## Setup

In [ ]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv, find_dotenv

dotenv_path = find_dotenv() or '/content/OPENAI_API_KEY.env'
load_dotenv(dotenv_path)

openai_api_key = os.getenv('OPENAI_API_KEY')

client = openai.OpenAI(api_key=openai_api_key)

In [ ]:
chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [ ]:
# Inspect chat_completion object
chat_completion

ChatCompletion(id='chatcmpl-Cc2HSsHDCQ6HHAsGzWktdNzlrduLy', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1763181266, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [ ]:
# Return the content of the first (and only) choice
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

## Agent

In [ ]:
class Agent:
  """
  A simple conversational agent that stores conversation history and interacts with a language model.

  Attributes:
      system (str): Optional system message to guide the agent's behavior.
      messages (list): Stores the conversation as a list of message dicts with roles and content.

  Methods:
      __call__(message): Sends a user message to the agent, runs the model, and returns the reply.
      execute(): Calls the OpenAI API with the full message history and returns the model's response.

  Usage:
      abot = Agent("You are a helpful assistant.")
      response = abot("Hello!")
  """
  def __init__(self, system=""): # is called automatically when we create an object
    # system: an optional system message that can guide the agent's behavior
    self.system = system

    # messages: stores the conversation history (role + content)
    self.messages = []

    # If a system message is provided, add it to the beginning of the conversation
    if self.system:
      self.messages.append({"role": "system", "content": system})

  def __call__(self, message): # runs when we do object(...)
    # When the agent is “called” with a user message, append it to the history
    # (the object is called like a function; adds user msg, runs execute(), stores reply)
    self.messages.append({"role": "user", "content": message})

    # Execute the agent to get a response from the model
    result = self.execute()

    # Append the assistant's response to the conversation history
    self.messages.append({"role": "assistant", "content": result})

    # Return the response text
    return result

  def execute(self):
    # Call the OpenAI API to get a completion using the conversation history
    completion = client.chat.completions.create(
        model="gpt-4o-mini", # (cheapest and most effective choice)
        temperature=0, # deterministic responses
        messages=self.messages) # the full message history
    return completion.choices[0].message.content

The class sets up its main attributes in ```__init__```, including the system prompt and the message history. The ```__call__``` method is triggered whenever we call the object like a function (e.g., abot("Hello")), and it adds the user message, runs the model through ```execute()```, saves the reply, and returns it. The ```execute()``` method handles the actual API call using the full conversation history.

In [ ]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [ ]:
# define the actions of the Agent
def calculate(what):
  return eval(what) # executes the string as Python code

def average_dog_weight(name):
  if name in "Scottish Terrier":
    return("Scottish Terriers average 20 lbs")
  elif name in "Border Collie":
    return("a Border Collies average weight is 37 lbs")
  elif name in "Toy Poodle":
    return("a toy poodles average weight is 7 lbs")
  else:
    return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [ ]:
# initialize the Agent with the given prompt
abot = Agent(prompt)

Inspect abot object.

In [ ]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'}]

In [ ]:
abot.system

'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'

In [ ]:
# abot.execute()

## Run the Agent manually (1)

In [ ]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the average weight of a Toy Poodle using the average_dog_weight action. 
Action: average_dog_weight: Toy Poodle
PAUSE


In [ ]:
# pass manually the Action needed to the function
result = average_dog_weight("Toy Poodle")

In [ ]:
result

'a toy poodles average weight is 7 lbs'

In [ ]:
# format the result as 'Observation' (=next prompt)
next_prompt = "Observation: {}".format(result)

In [ ]:
next_prompt

'Observation: a toy poodles average weight is 7 lbs'

In [ ]:
# and pass it again to abot to generate the final answer
abot(next_prompt)

'Answer: A Toy Poodle weighs approximately 7 lbs.'

Inspect abot's attributes now. Notice that all the manually above steps are included.

In [ ]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [ ]:
# abot.system

## Run the Agent manually (2)

In [ ]:
# re-initialize it
abot = Agent(prompt)

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. What is their combined weight"""
abot(question)

'Thought: I need to find the average weights of both a Border Collie and a Scottish Terrier to calculate their combined weight. \nAction: average_dog_weight: Border Collie\nPAUSE'

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [ ]:
abot(next_prompt)

'Thought: Now I need to find the average weight of a Scottish Terrier to complete the calculation for their combined weight. \nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [ ]:
abot(next_prompt)

'Thought: I have the average weights for both dogs: 37 lbs for the Border Collie and 20 lbs for the Scottish Terrier. I can now calculate their combined weight. \nAction: calculate: 37 + 20\nPAUSE'

In [ ]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [ ]:
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.'

In [ ]:
# see that all the steps we did manually are in messages
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user',
  'content': 'I have 2 dogs,

## Automated Agent loop

In [ ]:
# regex to extract the action name and its input from lines like: "Action: Search: query"
action_re = re.compile(r'^Action: (\w+): (.*)$') # python regular expression to selection action

In [ ]:
action_re

re.compile(r'^Action: (\w+): (.*)$', re.UNICODE)

```^Action:``` → line must start with “Action:”

```(\w+)``` → captures the action name (letters, digits, underscore)

```(.*)``` → captures everything after that (the action input)

**Example:**
```python
line = "Action: Search: cats and dogs"
match = action_re.match(line)

print(match.group(1)) # → Search
print(match.group(2)) # → cats and dogs
```

In [ ]:
# Automated Agent Loop
def query(question, max_turns=5):
  """
  Run an automated loop with the Agent to handle actions and observations.

  Parameters:
    question : str
        The initial user query or prompt to start the conversation.
    max_turns : int, optional (default=5)
        Maximum number of iterations the agent can take to respond and perform actions.
        Prevents infinite loops in case the agent keeps generating actions.

  Behavior:
    - Sends the initial question to the agent.
    - Checks the agent's response for any actions using `action_re`.
    - If an action is detected, executes it via `known_actions` and feeds the observation back to the agent as a new prompt.
    - Stops either when no actions are returned or when `max_turns` is reached.
    - Prints all agent responses and observations along the way.
  """
  i = 0
  bot = Agent(prompt) # create a fresh Agent for this query (using prompt as system message)
  next_prompt = question # first message sent to the agent (will be user's question)

  while i < max_turns:
    i += 1

    result = bot(next_prompt) # send prompt to agent, get model reply
    print(result)

    # detect lines that match "Action: <name>: <input>"
    actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

    if actions:
      # model wants to run an action
      action, action_input = actions[0].groups()

      # safety check for unknown actions
      if action not in known_actions:
        raise Exception("Unknown action: {}: {}".format(action, action_input))

      print(" -- running {} {}".format(action, action_input))

      # run the action function with the extracted input
      observation = known_actions[action](action_input)
      print("Observation:", observation)

      # feed the observation back to the agent so it can continue reasoning
      next_prompt = "Observation: {}".format(observation)
    else:
      # no action → final answer reached
      return

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. What is their combined weight"""
query(question)

Thought: I need to find the average weights of both a Border Collie and a Scottish Terrier to calculate their combined weight. 
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now I need to find the average weight of a Scottish Terrier to complete the calculation for their combined weight. 
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: I have the average weights for both dogs: a Border Collie weighs 37 lbs and a Scottish Terrier weighs 20 lbs. Now I can calculate their combined weight. 
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.
